In [1]:
## this notebook aims to prepare EARNINGS CALL files for transcripton. that includes
## - pre process to correct file formats/encoding 
## - take metadata about the audio
## - submit a job for transcribing 

In [2]:
# read audio file (full) -> upload from user or LINK 
# input meta for the file 
# transcribe 




In [3]:
# upload to s3 -> for dev purpose 'data' folder is s3 
# pull audio -> create meta -> start processing


## meta 
- filename 
- type 
- format 
- url 
- length of audio 
- org name 
- about org/convo [optional]
- number of speakers [opt]
- 

In [4]:
# note 
# transcript will be saved in the database as 'raw'.  post verification by user it is stored as final trx.
# embedding happens post user validation 
# embedding strategy is default - tbd 
## default - full file, chunk 

In [5]:
from enum import Enum
from typing import List 
from enums import tsFormats
class RecordingFormats(Enum):
    MP3 = 'mp3'
    WAV = 'wav'
    M4A = 'm4a'

class RecordingTypes(Enum):
   CONCALL = 'concall'
   INTERVIEW = 'interview'
   

class AudioFileMeta:
    def __init__(self, 
                 file_name:str,  #pk
                 input_file_format:RecordingFormats,
                 recording_type:RecordingTypes,
                 audio_length:float=None,
                 ) -> None:
        
        pass

class TranscriptMetadata:
    def __init__(self, 
                 audio_file_name:str,  #pk
                 transcript_format:tsFormats,
                 num_chunks:int=None,
                 org_name:str=None, 
                 audio_base_prompt:str=None,
                 speakers:dict=None
                 ) -> None:
        
        self.audio_file_name = audio_file_name
        self.transcript_format = transcript_format
        self.num_chunks = num_chunks
        self.org_name = org_name
        self.audio_base_prompt = audio_base_prompt
        self.speakers = speakers
        
class TranscriptChunk:
    def __init__(self,
                 id,
                 file_name,
                 text,
                 embedding=None) -> None:
        pass


In [13]:
from utils import get_openai_client
from utils_transcribe import transcribe_audio_in_format
from utils_dir import save_transcript_text_json
from utils_audio import chop_audio, convert_audio_to_ogg, open_audio_as_segment
from dirs import *
raw_dir = DATA_DIR
ogg_dir = PROCESSED_DIR
audio_file = 'earning_call_morepen.mp3'
audio_type = RecordingTypes.CONCALL
audio_format = RecordingFormats.MP3

class AudioTranscript:
    def __init__(self,
                 text,
                 meta:dict) -> None:
        self.text = text 
        self.meta = meta 

# the following is one atomic chain 
def read_audio_file(file_name:str):
    ogg_file = convert_audio_to_ogg(file_name=file_name)
    if not ogg_file:
        raise Exception("Unable to convert audio to OGG")
   
    audio = open_audio_as_segment(ogg_file)
    total_duration_milliseconds = len(audio)
    total_duration_seconds = total_duration_milliseconds / 1000
    meta = AudioFileMeta(file_name=file_name,
                         input_file_format=RecordingFormats.MP3,
                         recording_type=RecordingTypes.CONCALL,
                         audio_length=total_duration_seconds)
    

    return meta, audio

def insert_audio_meta(meta:AudioFileMeta):
    pass

def transcribe_audio_in_chunks(audio_path:Path,
                            base_prompt:str,
                            output_dir=TS_DIR): # save text of each chunk (audio chunk)
    
    """Finish transcription, concat transcripts and save in a AudioTranscript object to a json.
    ensure ogg file in full path is entered, base prompt to catch domain specific words.
        chop file and save chunks in chop dir. Transcribe each chunk sequentially reading from disk. 
        
    """
    meta = TranscriptMetadata(audio_file_name=audio_path.stem,
                              transcript_format=tsFormats.JSON)
    chunk_files = chop_audio(audio_path)
    meta.num_chunks = len(chunk_files)
    # start transcription of chunks, 
    client = get_openai_client()
    print("starting transcriptioni, got client")
    prompt = base_prompt
    output_prefix = 'conc_'
    id = 0
    transcript_final = AudioTranscript(text='',meta={'total_chunks':len(chunk_files),
                                                     'count':0 
                                                     }
                                            )

    print("chunk files:", type(chunk_files), chunk_files)
    for chunk_file in chunk_files:

        file_path = Path(CHOP_DIR,chunk_file)
        print("file path of chunk :", file_path.__str__())
        
        trx = transcribe_audio_in_format(client=client,
                                        audio_file_path= file_path ,
                                        prompt=prompt)
        prompt = trx.text #update prompt for next chunk 
        transcript_final.text += f'{trx.text} '
        transcript_final.meta['count'] += 1
        

    out_file_name =  f'{output_prefix}{audio_path.stem}'

    if save_transcript_text_json(transcript_final,
                         out_file_name,
                         dir=output_dir):

        print(id, "file processsed as ts: ", file_path.stem)
        return f'{out_file_name}.json'
            



## prepare a database to store the transcripts after processing chopped files 
## preferably faiss or mongo db ( embeddings will be stored there as well )

def insert_raw_transcript(chunk_meta):
    pass 


    

In [7]:
# read_audio_file(file_name=audio_file)


In [8]:

# ogg_file = 'earning_call_morepen.ogg'
# audio = open_audio_as_segment(ogg_file)
# audio

In [9]:
about_morepen = "Embarking on a transformative journey over four decades, Morepen has consistently strived to embody an organization marked by discernible, impactful differences. Every action we undertake is imbued with a profound commitment to enrich the lives we touch. Nestled amidst the serene surroundings of Baddi, our cutting-edge manufacturing facility boasts a scientifically integrated complex housing 10 plants, each tailored to produce specific product lines."

In [1]:
ogg_file = 'earning_call_morepen.ogg'
input_path = Path(PROCESSED_DIR,ogg_file)

output_file = transcribe_audio_in_chunks(base_prompt=about_morepen,audio_path=input_path,)
output_file

NameError: name 'Path' is not defined

In [ ]:
## massage raw transcripts 
### raw transcripts are just blobs of text they can be enriched by 
# - giving to user for corrections 
# - giving to gpt for summarisation, diarization, formatting, topic extractions etc 
